# Openai Proxy Quickstart - Enterprise Mode

<a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/llm_monitoring/openai_proxy_quickstart_enterprise_mode.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

A simple tool to capture metadata about your interactions with the OpenAI API.  Currently supports ChatCompletion endpoint, including streaming.

Fundamentally, you can directly request data from the proxy using any technique you prefer (eg. `curl`), or use the openai client library. This notebook will highlight both apporaches

In [ ]:
# if not already installed
!pip install -qqq openai

## Settings & Setup

The following setup the openai client to log correctly. Concretely, we need to:
1. Change your openai api base url.
2. Change your openai api key wandb api key.



In [ ]:
# Step 1: Configure target URL
WANDB_BASE_URL = # SET ME! - unlikely to be "https://api.wandb.ai"

In [ ]:
# Step 2: Ensure WANDB_API_KEY is set in env
import os
from getpass import getpass

WANDB_APP_URL = WANDB_BASE_URL if WANDB_BASE_URL != "https://api.wandb.ai" else "https://app.wandb.ai"
WEAVE_ROOT_URL = WANDB_BASE_URL + "/weave" if WANDB_BASE_URL != "https://api.wandb.ai" else "https://weave.wandb.ai"
if os.getenv("WANDB_API_KEY") is None:
    os.environ["WANDB_API_KEY"] = getpass(f"Paste your W&B key from: {WANDB_APP_URL}/authorize \n")
print("W&B API key configured")

In [ ]:
# Step 3: Perform setup
import openai
openai.api_base = f"{WANDB_BASE_URL}/proxy/openai/v1"
openai.api_key = os.getenv("WANDB_API_KEY")

## Start logging
You can use your existing API calls as-is, now with extra logging!

By default, this will log to `{enterprise_admin_entity}/monitoring/openai`, where the project is `monitoring` and the table name is `openai`.

### Advanced configuration
1. Optionally add custom attributes to your table with the `X-Wandb-Attribute-` prefix.  For example, adding the header `X-Wandb-Attribute-Foo: bar` will add a column `Foo` with the value `bar` to your table.
2. Optionally add the `X-Wandb-Client-Id` header if you want to further group related requests together.  By default, each request will have its own unique Client ID.


### Style 1: Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Tell me a joke about gradients as if you were Shakespeare"}],
    headers={
        # Optional Client ID
        "X-Wandb-Client-Id": "Shakespearean-Gradients-001",
        # Optional Key-Value pairs:
        "X-Wandb-Attribute-Persona": "Shakespeare",
    }
)
response

In [ ]:
print(f"Admins can view logs at {WEAVE_ROOT_URL}/browse/wandb/[ADMIN_ENTITY]/monitoring/openai")

### Style 2: Using CURL

In [ ]:
# This cell simply exposes the correct strings as env variables
# so that they can be used in the next cell to demonstrate curling
# the endpoint. These env vars are not required - it is just an easy
# way to pass data from the notebook to the bash command
os.environ["OPENAI_PROXY_API_BASE"] = openai.api_base
os.environ["OPENAI_PROXY_API_KEY"] = openai.api_key

In [ ]:
# Similar to the openai sdk example, the X-Wandb-Client-Id and X-Wandb-Attribute-[KEY] headers are optional

In [ ]:
%%bash
curl "$OPENAI_PROXY_API_BASE/chat/completions" \
-H "Authorization: Bearer $OPENAI_PROXY_API_KEY" \
-H "Content-Type: application/json" \
-H "X-Wandb-Client-Id: Shakespearean-Gradients-001" \
-H "X-Wandb-Attribute-Persona: Shakespeare" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions if you were Shakespeare!"}]
   }'

In [ ]:
print(f"Admins can view logs at {WEAVE_ROOT_URL}/browse/wandb/[ADMIN_ENTITY]/monitoring/openai")

## Use Proxy with Weave SDK

Now, users will typically want to track their own logs so they can analyze them independently. In this case we combine the monitoring techiques discussed in [OpenAI Monitoring Demo](./openai_client_quickstart.ipynb) with the proxy concepts here.

You can use this proxy with the Weave SDK to log out to a personal streamtable and a proxy-configured streamtable simultaneously.  This can be useful for admins who want to monitor API calls in their organization.
- Users specify a personal table path with `init_monitor(entity/project/table)`.  Any `ChatCompletion.create` calls with automatically log to this table.
- Then, if the `openai.api_base` and `openai.api_key` are set to proxy format, the above calls will also log the proxy-configured table.

NOTE: This currently does not work with streaming `ChatCompletion`, but we hope to support it soon!

In [ ]:
WB_PROJECT = "llmon"
STREAM_NAME = "openai_logs"
WB_ENTITY = ""  # optional: wandb username or team name

**Note**: The WB_ENTITY will match the default entity associated with your [W&B API key](wandb.ai/authorize). You can optionally set WB_ENTITY to a different wandb username or team name. Log in to W&B and navigate to [the Home Page](https://wandb.ai/home) to see any other valid options for your WB_ENTITY under your "Profile" and "Teams" in the left sidebar.

In [ ]:
!pip install weave tiktoken

In [ ]:
import os
os.environ["WANDB_BASE_URL"] = WANDB_BASE_URL
import wandb
wandb.login()

In [ ]:
import openai
from weave.legacy.monitoring import openai as openaimon
from weave.legacy.monitoring import init_monitor

m = init_monitor(f"{WB_ENTITY}/{WB_PROJECT}/{STREAM_NAME}")

r = openaimon.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"hello world!"}])
print(r)
r = openaimon.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"what is 2+2?"}])
print(r)